### PYTHON CODE TO EXTRACT BOARDING PASS INFORMATION

#### Import required Libraries

In [11]:
import os, json
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.storage.blob import (
    BlobSasPermissions,
    generate_blob_sas,
    BlobServiceClient
)
from datetime import datetime, timedelta

#### Azure authentication

In [ ]:
endpoint = "https://document-intelligence-udacity.cognitiveservices.azure.com/"
key      = "YOUR_AZURE_FORM_RECOGNIZER_KEY"
client   = DocumentAnalysisClient(endpoint, AzureKeyCredential(key))
print(client)

#### Azure authentication Blob storage and SAS creation

In [ ]:
account_name = "001finalproject"
account_key = "YOUR_AZURE_STORAGE_ACCOUNT_KEY"
container_name = "step2"
blob_name = "boarding-Alberto-Leon.pdf"

In [14]:
# SAS expliry time
sas_expiry = datetime.utcnow() + timedelta(hours=1)

# URL base
blob_url_base = f"https://{account_name}.blob.core.windows.net/{container_name}/{blob_name}"

# token generation
sas_token = generate_blob_sas(
    account_name=account_name,
    container_name=container_name,
    blob_name=blob_name,
    account_key=account_key,
    permission=BlobSasPermissions(read=True),
    expiry=sas_expiry
)

# Construir la URL completa
blob_url_with_sas = f"{blob_url_base}?{sas_token}"

#### Analysis

In [16]:
# 1) Analyze the document 
poller  = client.begin_analyze_document_from_url(
            model_id="Boarding_pass_data_extractor",   
            document_url=blob_url_with_sas,
            locale="en-US"                    
          )

result = poller.result()

# 2) Print the results

for doc in result.documents:
    print("Extracted boarding pass fields:")
    for field, value in doc.fields.items():
        print(f"{field}: {value.value} (confidence: {value.confidence:.2f})")


Extracted boarding pass fields:
PassengerName: Alberto León (confidence: 0.94)
Carrier: UA (confidence: 0.96)
Flight No.: 234 (confidence: 0.97)
Class: E (confidence: 0.96)
From: San Francisco (confidence: 0.95)
To: Chicago (confidence: 0.97)
Seat: 20B (confidence: 0.97)
Date: April 20, 2022 (confidence: 0.94)
Boarding Time: 10:00 AM PST (confidence: 0.91)
Gate: G1 (confidence: 0.97)
Baggage: NO (confidence: 0.96)
